In [1]:
import numpy as np

# scipy.special for the sigmoid function expit()
from scipy.special import expit

import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
%matplotlib inline

In [2]:
# helper to load data from PNG image files
import scipy.misc

# helper to rotating image arrays
from scipy.ndimage.interpolation import rotate

In [3]:
np.random.seed(42)

In [4]:
# neural network class definition
class NeuralNetwork:
    
    # initialize the neural network
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = input_nodes
        self.hnodes = hidden_nodes
        self.onodes = output_nodes
        
        # learning rate
        self.lr = learning_rate
        
        # link weight matrices, wih(Weight for input_hidden) and who(Weight for hidden_output)
        # weight inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc
        self.wih = np.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))  # hnodes x inodes matrix
        self.who = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))  # onodes x hnodes matrix
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: expit(x)
        
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = np.array(inputs_list, ndmin=2).T   # inodes x 1 vector
        targets = np.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        # X_hidden = W_input_hidden * I
        hidden_inputs = np.dot(self.wih, inputs)
        
        # calculate the signals emerging from hidden layer
        # O_hidden = sigmoid(X_hidden)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final ouput layer
        # X_output = W_hidden_output * O_hidden
        final_inputs = np.dot(self.who, hidden_outputs)
        
        # calculate the signals emerging from final output layer
        # O_output = sigmoid(X_output)
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = np.dot(self.who.T, output_errors)
        
        # update the weight for the links between the hidden and output layers
        self.who += self.lr * np.dot((output_errors * final_outputs * (1.0 - final_outputs)), np.transpose(hidden_outputs))
        
        # update the weight for the links between the input and hidden layers
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), np.transpose(inputs))
        
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = np.array(inputs_list, ndmin=2).T   # inodes x 1 vector
        
        # calculate signals into hidden layer
        # X_hidden = W_input_hidden * I
        hidden_inputs = np.dot(self.wih, inputs)
        
        # calculate the signals emerging from hidden layer
        # O_hidden = sigmoid(X_hidden)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final ouput layer
        # X_output = W_hidden_output * O_hidden
        final_inputs = np.dot(self.who, hidden_outputs)
        
        # calculate the signals emerging from final output layer
        # O_output = sigmoid(X_output)
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs
    
    # save neural network weight
    def save(self, filename):
        with open(filename, 'wb') as fobj:            
            np.save(fobj, self.wih)
            np.save(fobj, self.who)
    
    # load neural network weight
    def load(self, filename):
        with open(filename, 'rb') as fobj: 
            self.wih = np.load(fobj)
            self.who = np.load(fobj)

In [5]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

# learning rate
learning_rate = 0.01

# create instance of neural network
n = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [12]:
# load trained neural network weight
n.load('trained_model/saved_wights.npy')

In [8]:
# load the mnist test data CSV file into a list
with open('mnist_dataset/mnist_test.csv') as fobj:
    test_data_list = fobj.readlines()

In [9]:
# test the neural network

# scorecard for how well the network performs, initially empty
scorecard = []

# go through all records in the test data set
for record in test_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')

    # correct answer is first value
    correct_label = int(all_values[0])

    # scale and shift the inputs [0.01, 1]
    inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01

    # query the network
    outputs = n.query(inputs)

    # the index of the highest value corresponds to the label
    label = np.argmax(outputs)

    # append correct or incorrect to list
    if (label == correct_label):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)

In [10]:
# calculate the performance score, the fraction of correct answers
scorecard_array = np.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)

performance =  0.975
